### 特征匹配

核心：
- 通过局部特征的相似性建立对应关系。

本质：
局部特征对比: 特征匹配关注的是图像中的关键点（如边缘、角点）周围的局部纹理，通过数学描述符（如SIFT的128维向量）表达这些区域的特征，而非比较整个图像内容。示例：两图中不同角度拍摄的同一建筑，即使光照变化、窗口的角点描述符可能相似，从而匹配成功。

特征匹配是通过局部特获赠的数学相似性建立对应关系的技术，它服务于更高层的视觉任务（如定位、重建），但无法直接理解图像语义。

####  Brute-Force蛮力匹配

In [1]:
import cv2 
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
img1 = cv2.imread('box.png', 0)
img2 = cv2.imread('box_in_scene.png', 0)

In [3]:
def cv_show(name,img):
    cv2.imshow(name, img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()

In [4]:
cv_show('img1',img1)

In [5]:
cv_show('img2',img2)

In [6]:
# 创建SIFT检测器
sift = cv2.SIFT_create()

In [7]:
# 检测关键点
kp1, des1 = sift.detectAndCompute(img1, None)
kp2, des2 = sift.detectAndCompute(img2, None)

### 1对1的匹配

In [8]:
# 使用Brute-Force(BF)特征匹配器，在两张图像（img1和img2）的特征描述符（des1和des2）之间进行双向交叉验证的1对1匹配，
# 筛选出最佳匹配结果并可视化前10个匹配点。

# 初始化Brute-Force匹配器
# 作用：创建暴力匹配器对象，通过逐个比较特征向量找到最佳匹配。
# crossCheck：开启双向交叉验证，表示两个特征点要互相匹，例如A中的第i个特征点与B中的第j个特征点最近的，并且B中的第j个特征点到A中的第i个特征点也是（匹配的对称性）。
# 隐藏函数：normType=cv2.NORM_L2（使用欧几里得距离[L2范数]计算特征向量差异）。
bf = cv2.BFMatcher(crossCheck=True)
# 执行特征匹配
# 输入：des1(查询描述符)和des2(训练描述符)。
# 输出：matches是cv2.DMatch对象的列表，每个对象包含以下信息：
# .queryIdx：查询图像（des1）中的特征点索引。
# .trainIdx：训练图像（des2）中的特征点索引。
# .distance：两个描述符之间的距离值（越小表示越相似）。
matches = bf.match(des1, des2)
# 按匹配质量排序
# 根据 .distance 从小到大排序，前10个为最佳匹配。
# 排序后matches[0]是距离最小的最佳匹配对。
matches = sorted(matches, key=lambda x: x.distance)

In [9]:
# 可视化前10个最佳匹配.
# img1, kp1：第一张图像及其特征点。
# img2, kp2：第二章图像及其特征点。
# matches[:10]：取前10个最佳匹配对。
# flags=2：绘制模式，cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS 表示不显示未匹配的点。
# 输出：img3是将两幅图片水平拼接并在匹配点间绘制连线的结果图。
img3 = cv2.drawMatches(img1, kp1, img2, kp2, matches[:10], None,flags=2)

In [10]:
cv_show('img3',img3)

### k对最佳匹配

In [12]:
# 使用Brute-Force(BF)匹配器结合KNN搜索与Lowe's比率测试，实现特征匹配的筛选和可视化，核心目标是提升匹配的铝棒性。

# 初始化Brute-Force匹配器
bf = cv2.BFMatcher()

# kNN 匹配搜索
# 作用：对每个查询特征点（des1中的点），在训练集（des2）中搜索2个最近邻匹配（k=2）。
# 输出：matches是一个列表，每个元素包含两个DMatch对象（最近邻m和次近邻n）。
matches = bf.knnMatch(des1, des2, k=2)

In [13]:
# Lowe's比率测试过滤
# 若【最近邻距离 < 0.75 * 次近邻距离】，则认为匹配可靠（避免相似特征导致的歧义问题）。
# 阈值选择：比率阈值常取0.7~0.8，需根据场景调整。较高阈值（如0.8）保留更多匹配但可能含噪声；较低阈值（如0.6）更严格但可能漏真。
good = []
for m, n in matches:
    if m.distance < 0.75 * n.distance:
        good.append([m])

In [14]:
# 可视化筛选后的匹配
# good中的每个元素必须是DMatch对象的列表（即使仅含一个匹配），因此使用[m]包裹。
# flags=2：绘制模式，cv2.DrawMatchesFlags_NOT_DRAW_SINGLE_POINTS 表示不显示未匹配的点。
# 输出：img3是将两幅图片水平拼接并在匹配点间绘制连线的结果图。
img3 = cv2.drawMatchesKnn(img1,kp1,img2,kp2,good,None,flags=2)

In [15]:
cv_show('img3',img3)

### cv2.FlannBasedMatcher

如果需要更快速完成操作，可以尝试使cv2.FlannBasedMatcher。

概念：
- cv2.FlannBasedMatcher 是 OpenCV 中基于 FLANN（Fast Library for Approximate Nearest Neighbors） 实现的特征匹配器，专为高效的大规模特征匹配设计。与暴力匹配（BFMatcher）相比，它通过近似最近邻搜索算法，在牺牲少量精度的情况下显著提升匹配速度，尤其适合处理高维数据（如 SIFT、SURF 等特征）。

核心：
- 近似最近邻（ANN）：FLANN不是严格找到最近邻，而是找到近似最优解。通过构建索引结构（如KD-Tree、LSH等），在搜索时跳过部分计算，速度比暴力匹配快10~100倍。
- 分两步操作：
  - 构建索引：根据特征描述符构建高效饿数据结构（如KD-Tree）。
  - 搜索匹配：利用索引快速查找最近邻，通过启发式策略限制搜索深度。

### 随机抽样一致算法（Random sample consensus，RANSAC）

RANSAC是一种鲁棒性的模型拟合算法，意旨从包含大量异常值的数据中，**高效估计出最佳数学模型参数**。

算法步骤：
1. 随机抽样：从数据中随机选取生成模型所需的最好样本量（n点）。示例：单应矩阵需要4个对点，平面拟合需要3点。
2. 模型生成：用选取的点计算模型参数（如最小二乘法、直接线性变换）。
3. 内点评估：每个数据点，计算其与模型的误差，保留误差小于阈值（t）的点作为内点。
4. 更新最优模型：若当前内点数超过历史最优值，更新最佳模型及内点集合。
5. 动态调整迭代次数（可选）：根据当前最大内点数，利用公式调整所需总迭代次数。
6. 最终模型优化：使用所有内点重新拟合模型（如加权最小二乘），提升精度。

![title](ransac_1.png)

选择初始样本点进行拟合，给定一个容忍范围，不断进行迭代

![title](ransac_2.png)

每一次拟合后，容差范围内都有对应的数据点数，找出数据点个数最多的情况，就是最终的拟合结果

![title](ransac_3.png)

#### 单应性矩阵

概念：
- 单应性矩阵式计算机视觉中的一个核心概念，用于描述两个平面图像之间的**投影变换关系**。它可以将一张图片上的点精确映射到另一张图片的对应位置，即使两图之间存在旋转、平移、缩放或倾斜的视角差异。通俗来说，他式让两张不同角度拍摄的“同一平面物体”对齐的数学工具。

直观例子：
假设你从正面和斜上方两个角度拍摄一本书的封面：
1. 正视图：封面式一个规整的矩形。
2. 斜视图：封面在照片中式一个梯形（近大远小）。
   单应性矩阵的作用，就是将斜视图中的梯形封面，“矫正”回正视图中的矩形，就像魔法一样对齐两张图片中的同一个平面。

数学形式：
单应性矩阵是一个3x3，如下的H矩阵。
它通过对齐坐标变换，将原图点(x,y)映射到目标图(x',y')，如下：
其中（x,y）和（x',y'）是两种图片中计算出来的对应点（至少需要4对），
计算出单应性3x3矩阵，将该矩阵应用到整个图片中完成最终匹配。

![title](ransac_4.png)

In [ ]:
单应性矩阵使用场景：
1. 视角矫正：将倾斜拍摄的平面物体（如证件、海报）拉正。
2. 图像拼接：对齐多张图片中的重叠区域，合成全景图。
3. 增强现实：将虚拟物体贴合到现实世界的平面（如地面、桌面）。
4. 平面跟踪：实时跟踪视频中动态变化的平面区域。